## Solving Sequence Problems with LSTM in Keras
<a href="https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras/">Tutorial Link </a>

In this article, you will learn how to perform time series forecasting that is used to solve sequence problems. <b/>

Time series forecasting refers to the type of problems where we have to predict an outcome based on time dependent inputs. A typical example of time series data is stock market data where stock prices change with time. Similarly, the hourly temperature of a particular place also changes and can also be considered as time series data. Time series data is basically a sequence of data, hence time series problems are often referred to as sequence problems. <b/>

Recurrent Neural Networks (RNN) have been proven to efficiently solve sequence problems. Particularly, Long Short Term Memory Network (LSTM), which is a variation of RNN, is currently being used in a variety of domains to solve sequence problems.

## Types of Sequence Problems
<p>Sequence problems can be broadly categorized into the following categories:</p>

* One-to-One: Where there is one input and one output. Typical example of a one-to-one sequence problems is the case where you have an image and you want to predict a single label for the image.

* Many-to-One: In many-to-one sequence problems, we have a sequence of data as input and we have to predict a single output. Text classification is a prime example of many-to-one sequence problems where we have an input sequence of words and we want to predict a single output tag.

* One-to-Many: In one-to-many sequence problems, we have single input and a sequence of outputs. A typical example is an image and its corresponding description.

* Many-to-Many: Many-to-many sequence problems involve a sequence input and a sequence output. For instance, stock prices of 7 days as input and stock prices of next 7 days as outputs. Chatbots are also an example of many-to-many sequence problems where a text sequence is an input and another text sequence is the output.

## One-to-One Sequence Problems
As I said earlier, in one-to-one sequence problems, there is a single input and a single output. In this section we will see two types of sequence problems. First we will see how to solve one-to-one sequence problems with a single feature and then we will see how to solve one-to-one sequence problems with multiple features.

## One-to-One Sequence Problems with a Single Feature
In this section, we will see how to solve one-to-one sequence problem where each time-step has a single feature.Let's first import the required libraries that we are going to use in this article:

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM, GlobalMaxPool1D
from tensorflow.keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Creating the Dataset
In this next step, we will prepare the dataset that we are going to use for this section.

In [65]:
X = list()
Y = list()
X = [ x + 1 for x in range(20) ]
Y = [ y * 15 for y in X ]
Y = np.array(Y)

print(X)
print(Y)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[ 15  30  45  60  75  90 105 120 135 150 165 180 195 210 225 240 255 270
 285 300]


In the script above, we create 20 inputs and 20 outputs. Each input consists of one time-step, which in turn contains a single feature. Each output value is 15 times the corresponding input value. If you run the above script, you should see the input and output values as shown above:

The input to LSTM layer should be in 3D shape i.e. (samples, time-steps, features). The samples are the number of samples in the input data. We have 20 samples in the input. The time-steps is the number of time-steps per sample. We have 1 time-step. Finally, features correspond to the number of features per time-step. We have one feature per time-step.

We can reshape our data via the following command:

In [66]:
X = array(X).reshape(len(X), 1, 1)
X.shape

(20, 1, 1)

## Solution via Simple LSTM
Now we can create our simple LSTM model with one LSTM layer.

In [41]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
None


<b> Let's now train our model:</b>

In [42]:
model.fit(X, Y, epochs=2000, validation_split=0.2, batch_size=5)

Epoch 1/2000
4/4 [==============================] - 0s 34ms/step - loss: 21001.5254 - val_loss: 77125.8281
Epoch 2/2000
4/4 [==============================] - 0s 5ms/step - loss: 20986.0996 - val_loss: 77077.0312
Epoch 3/2000
4/4 [==============================] - 0s 5ms/step - loss: 20972.5430 - val_loss: 77024.7891
Epoch 4/2000
4/4 [==============================] - 0s 5ms/step - loss: 20955.8867 - val_loss: 76972.5391
Epoch 5/2000
4/4 [==============================] - 0s 5ms/step - loss: 20940.9746 - val_loss: 76914.7031
Epoch 6/2000
4/4 [==============================] - 0s 5ms/step - loss: 20922.6699 - val_loss: 76854.2656
Epoch 7/2000
4/4 [==============================] - 0s 5ms/step - loss: 20905.7969 - val_loss: 76790.9219
Epoch 8/2000
4/4 [==============================] - 0s 5ms/step - loss: 20887.7188 - val_loss: 76722.8281
Epoch 9/2000
4/4 [==============================] - 0s 5ms/step - loss: 20868.6230 - val_loss: 76651.4141
Epoch 10/2000
4/4 [==========================

We train our model for 2000 epochs with a batch size of 5. You can choose any number. Once the model is trained, we can make predictions on a new instance.

Let's say we want to predict the output for an input of 30. The actual output should be 30 x 15 = 450. Let's see what value do we get. First, we need to convert our test data to the right shape i.e. 3D shape, as expected by LSTM. The following script predicts the output for the number 30:

In [43]:
test_input = array([30])
test_input = test_input.reshape(len(test_input), 1, 1)
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[442.80066]]


I got an output value of 442.80066 which is slightly less than 450.

Note: It is important to mention that the outputs that you obtain by running the scripts will different from mine. This is because the LSTM neural network initializes weights with random values and your values. But overall, the results should not differ much.

## Solution via Stacked LSTM
Let's now create a stacked LSTM and see if we can get better results. The dataset will remain the same, the model will be changed. Look at the following script:

In [44]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 1, 50)             10400     
_________________________________________________________________
lstm_9 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________
None


In the above model, we have two LSTM layers. Notice, the first LSTM layer has parameter return_sequences, which is set to True. When the return sequence is set to True, the output of the hidden state of each neuron is used as an input to the next LSTM layer. The summary of the above model is as follows:

<b>Next, we need to train our model as shown in the following script:</b>

In [45]:
history = model.fit(X, np.array(Y) , epochs=2000, validation_split=0.2, verbose=1, batch_size=5)

Epoch 1/2000
4/4 [==============================] - 0s 50ms/step - loss: 21034.2031 - val_loss: 77253.5938
Epoch 2/2000
4/4 [==============================] - 0s 6ms/step - loss: 21022.2188 - val_loss: 77214.8359
Epoch 3/2000
4/4 [==============================] - 0s 5ms/step - loss: 21009.9336 - val_loss: 77174.3984
Epoch 4/2000
4/4 [==============================] - 0s 5ms/step - loss: 20998.7168 - val_loss: 77127.2031
Epoch 5/2000
4/4 [==============================] - 0s 5ms/step - loss: 20984.8438 - val_loss: 77073.0625
Epoch 6/2000
4/4 [==============================] - 0s 7ms/step - loss: 20970.1309 - val_loss: 77003.5781
Epoch 7/2000
4/4 [==============================] - 0s 6ms/step - loss: 20951.1250 - val_loss: 76912.4453
Epoch 8/2000
4/4 [==============================] - 0s 6ms/step - loss: 20924.0312 - val_loss: 76797.9375
Epoch 9/2000
4/4 [==============================] - 0s 6ms/step - loss: 20896.3418 - val_loss: 76646.3828
Epoch 10/2000
4/4 [==========================

Once the model is trained, we will again make predictions on the test data point i.e. 30.

In [47]:
test_input = array([30])
test_input = test_input.reshape(len(test_input), 1, 1)
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[452.7035]]


I got an output of 452.7035 which is better than 437, the number that we achieved via single LSTM layer.

## One-to-One Sequence Problems with Multiple Features

In the last section, each input sample had one time-step, where each time-step had one feature. In this section we will see how to solve one-to-one sequence problem where input time-steps have multiple features.

## Creating the Dataset
Let's first create our dataset. Look at the following script:

In [2]:
nums = 25
X1 = list()
X2 = list()

X = list()
Y = list()

X1 = [ (x + 1)*2 for x in range(25) ]
X2 = [ (x + 1)*3 for x in range(25) ]
Y = np.array([ x1*x2 for x1, x2 in zip(X1,X2) ])

print(X1)
print(X2)
print(Y)

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75]
[   6   24   54   96  150  216  294  384  486  600  726  864 1014 1176
 1350 1536 1734 1944 2166 2400 2646 2904 3174 3456 3750]


In the script above, we create three lists: X1, X2, and Y. Each list has 25 elements, which means that that the total sample size is 25. Finally, Y contains the output. X1, X2, and Y lists have been printed below:

Each element in the output list, is basically the product of the corresponding elements in the <b>X1</b> and <b>X2</b>  lists. For instance, the second element in the output list is 24, which is the product of the second element in list X1 i.e. 4, and the second element in the list X2 i.e. 6.

<b>The input will consist of the combination of X1 and X2 lists, where each list will be represented as a column. The following script creates the final input:</b>

In [3]:
X = np.column_stack((X1, X2))
print(X.shape)

(25, 2)


Here the X variable contains our final feature set. You can see it contains two columns i.e. two features per input. As we discussed earlier, we need to convert the input into 3-dimensional shape. Our input has 25 samples, where each sample consist of 1 time-step and each time-step consists of 2 features. The following script reshapes the input.

In [4]:
X = array(X).reshape(len(X), 1, 2)
X.shape

(25, 1, 2)

## Solution via Simple LSTM
We are now ready to train our LSTM models. Let's first develop a single LSTM layer model as we did in the previous section:

In [6]:
model = Sequential()
model.add(LSTM(80, activation="relu", input_shape=(1, 2)))
model.add(Dense(10, activation="relu"))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 80)                26560     
_________________________________________________________________
dense (Dense)                (None, 10)                810       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 27,381
Trainable params: 27,381
Non-trainable params: 0
_________________________________________________________________
None


Here our LSTM layer contains 80 neurons. We have two dense layers where first layer contains 10 neurons and the second dense layer, which also acts as the output layer, contains 1 neuron. The summary of the model is above:

In [7]:
model.fit(X, Y, epochs=2000, validation_split=0.2, batch_size=5)

Epoch 1/2000
4/4 [==============================] - 0s 37ms/step - loss: 1301724.3750 - val_loss: 10303413.0000
Epoch 2/2000
4/4 [==============================] - 0s 4ms/step - loss: 1300532.3750 - val_loss: 10296226.0000
Epoch 3/2000
4/4 [==============================] - 0s 4ms/step - loss: 1299294.6250 - val_loss: 10287930.0000
Epoch 4/2000
4/4 [==============================] - 0s 5ms/step - loss: 1297828.3750 - val_loss: 10279877.0000
Epoch 5/2000
4/4 [==============================] - 0s 6ms/step - loss: 1296414.7500 - val_loss: 10271869.0000
Epoch 6/2000
4/4 [==============================] - 0s 4ms/step - loss: 1295118.6250 - val_loss: 10265966.0000
Epoch 7/2000
4/4 [==============================] - 0s 4ms/step - loss: 1294024.0000 - val_loss: 10259580.0000
Epoch 8/2000
4/4 [==============================] - 0s 4ms/step - loss: 1292713.7500 - val_loss: 10252686.0000
Epoch 9/2000
4/4 [==============================] - 0s 4ms/step - loss: 1291368.7500 - val_loss: 10244544.0000


Let's test our trained model on a new data point. Our data point will have two features i.e. (55,80) the actual output should be 55 x 80 = 4400. Let's see what our algorithm predicts. Execute the following script:

I got 3943.8516 in the output, which is far from the actual output.

In [13]:
test_input = array([55,80])
test_input = test_input.reshape((1, 1, 2))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[3814.098]]


## Solution via Stacked LSTM
Let's now create a more complex LSTM with multiple LSTM and dense layers and see if we can improve our answer:

In [9]:
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(1,2)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 200)            162400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 100)            120400    
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 50)             30200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 25)                7600      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                520       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

<b>The model summary is as follows:</b>

The next step is to train our model and test it on the test data point i.e. (55,80).

To improve the accuracy, we will reduce the batch size, and since our model is more complex now we can also reduce the number of epochs. The following script trains the LSTM model and makes prediction on the test datapoint.

In [10]:
history = model.fit(X, Y, epochs=1000, validation_split=0.1, verbose=1, batch_size=3)

Epoch 1/1000
8/8 [==============================] - 0s 49ms/step - loss: 1884077.5000 - val_loss: 12026313.0000
Epoch 2/1000
8/8 [==============================] - 0s 7ms/step - loss: 1883853.6250 - val_loss: 12024103.0000
Epoch 3/1000
8/8 [==============================] - 0s 7ms/step - loss: 1883228.3750 - val_loss: 12007005.0000
Epoch 4/1000
8/8 [==============================] - 0s 7ms/step - loss: 1876328.7500 - val_loss: 11904460.0000
Epoch 5/1000
8/8 [==============================] - 0s 7ms/step - loss: 1835786.0000 - val_loss: 11567787.0000
Epoch 6/1000
8/8 [==============================] - 0s 7ms/step - loss: 1736436.7500 - val_loss: 10572895.0000
Epoch 7/1000
8/8 [==============================] - 0s 7ms/step - loss: 1457425.1250 - val_loss: 8215996.5000
Epoch 8/1000
8/8 [==============================] - 0s 7ms/step - loss: 827759.0625 - val_loss: 4368990.5000
Epoch 9/1000
8/8 [==============================] - 0s 7ms/step - loss: 262310.7812 - val_loss: 665631.2500
Epoch 

In [14]:
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[3814.098]]


In the output, I got a value of 3814.098 which is still less than 4400, but is much better than the previously obtained value of 3263.44 using single LSTM layer. You can play with different combination of LSTM layers, dense layers, batch size and the number of epochs to see if you get better results.

## Many-to-One Sequence Problems
In the previous sections we saw how to solve one-to-one sequence problems with LSTM. In a one-to-one sequence problem, each sample consists of single time-step of one or multiple features. Data with single time-step cannot be considered sequence data in a real sense. Densely connected neural networks have been proven to perform better with single time-step data.

Real sequence data consists of multiple time-steps, such as stock market prices of past 7 days, a sentence containing multiple words, and so on.

In this section, we will see how to solve many-to-one sequence problems. In many-to-one sequence problems, each input sample has more than one time-step, however the output consists of a single element. Each time-step in the input can have one or more features. We will start with many-to-one sequence problems having one feature, and then we will see how to solve many-to-one problems where input time-steps have multiple features.

## Many-to-One Sequence Problems with a Single Feature
Let's first create the dataset. Our dataset will consist of 15 samples. Each sample will have 3 time-steps where each time-step will consist of a single feature i.e. a number. The output for each sample will be the sum of the numbers in each of the three time-steps. For instance, if our sample contains a sequence 4,5,6 the output will be 4 + 5 + 6 = 10.

## Creating the Dataset
Let's first create a list of integers from 1 to 45. Since we want 15 samples in our dataset, we will reshape the list of integers containing the first 45 integers.

In [16]:
X = np.array([x+1 for x in range(45)])
print(X)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]


We can reshape it into number of samples, time-steps and features using the following function:
The above script converts the list X into 3-dimensional shape with 15 samples, 3 time-steps, and 1 feature. The script above also prints the reshaped data.

In [25]:
X = array(X).reshape(15, 3, 1)
print(X[0])
print(X.shape)

[[1]
 [2]
 [3]]
(15, 3, 1)


We have converted our input data into the right format, let's now create our output vector. As I said earlier, each element in the output will be equal to the sum of the values in the time-steps in the corresponding input sample. The following script creates the output vector:

In [26]:
Y = list()
for x in X:
    Y.append(x.sum())

Y = np.array(Y)
print(Y)

[  6  15  24  33  42  51  60  69  78  87  96 105 114 123 132]


## Solution via Simple LSTM
Let's now create our model with one LSTM layer.

In [27]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [28]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 0s 127ms/step - loss: 4297.9751 - val_loss: 16183.9658
Epoch 2/1000
1/1 [==============================] - 0s 14ms/step - loss: 4284.1870 - val_loss: 16129.5967
Epoch 3/1000
1/1 [==============================] - 0s 14ms/step - loss: 4270.3149 - val_loss: 16075.3594
Epoch 4/1000
1/1 [==============================] - 0s 13ms/step - loss: 4256.5098 - val_loss: 16021.2080
Epoch 5/1000
1/1 [==============================] - 0s 17ms/step - loss: 4242.8618 - val_loss: 15968.1885
Epoch 6/1000
1/1 [==============================] - 0s 13ms/step - loss: 4229.5493 - val_loss: 15917.6455
Epoch 7/1000
1/1 [==============================] - 0s 13ms/step - loss: 4216.1616 - val_loss: 15866.6641
Epoch 8/1000
1/1 [==============================] - 0s 14ms/step - loss: 4202.6074 - val_loss: 15815.1357
Epoch 9/1000
1/1 [==============================] - 0s 16ms/step - loss: 4188.8535 - val_loss: 15762.9424
Epoch 10/1000
1/1 [==========================

Once the model is trained, we can use it to make predictions on the test data points. Let's predict the output for the number sequence 50,51,52. The actual output should be 50 + 51 + 52 = 153. The following script converts our test points into a 3-dimensional shape and then predicts the output:

In [36]:
test_input = array([50,51,52])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(f"Actual: 153 || Predic: {test_output}")

Actual: 153 || Predic: [[151.82193]]


## Solution via Stacked LSTM
Let's now create a complex LSTM model with multiple layers and see if we can get better results. Execute the following script to create and train a complex model with multiple LSTM and dense layers:

In [37]:
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(3, 1)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [38]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 0s 348ms/step - loss: 4045.5496 - val_loss: 15180.0205
Epoch 2/1000
1/1 [==============================] - 0s 15ms/step - loss: 4044.5833 - val_loss: 15177.8604
Epoch 3/1000
1/1 [==============================] - 0s 15ms/step - loss: 4043.9434 - val_loss: 15175.2812
Epoch 4/1000
1/1 [==============================] - 0s 15ms/step - loss: 4043.1699 - val_loss: 15172.0078
Epoch 5/1000
1/1 [==============================] - 0s 15ms/step - loss: 4042.2458 - val_loss: 15167.6299
Epoch 6/1000
1/1 [==============================] - 0s 15ms/step - loss: 4041.1101 - val_loss: 15161.5107
Epoch 7/1000
1/1 [==============================] - 0s 16ms/step - loss: 4039.6543 - val_loss: 15152.4873
Epoch 8/1000
1/1 [==============================] - 0s 16ms/step - loss: 4037.7148 - val_loss: 15138.3945
Epoch 9/1000
1/1 [==============================] - 0s 19ms/step - loss: 4035.0361 - val_loss: 15117.2373
Epoch 10/1000
1/1 [==========================

In [39]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: 153 || Predic: {test_output}")

Actual: 153 || Predic: [[149.8522]]


## Solution via Bidirectional LSTM
Bidirectional LSTM is a type of LSTM which learns from the input sequence from both forward and backward directions. The final sequence interpretation is the concatenation of both forward and backward learning passes. Let's see if we can get better results with bidirectional LSTMs.

The following script creates a bidirectional LSTM model with one bidirectional layer and one dense layer which acts as the output of the model.

In [40]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [41]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 0s 196ms/step - loss: 3772.4753 - val_loss: 13928.2627
Epoch 2/1000
1/1 [==============================] - 0s 15ms/step - loss: 3736.5217 - val_loss: 13785.1992
Epoch 3/1000
1/1 [==============================] - 0s 15ms/step - loss: 3700.3318 - val_loss: 13641.1650
Epoch 4/1000
1/1 [==============================] - 0s 15ms/step - loss: 3663.7415 - val_loss: 13495.8525
Epoch 5/1000
1/1 [==============================] - 0s 16ms/step - loss: 3626.4355 - val_loss: 13347.3428
Epoch 6/1000
1/1 [==============================] - 0s 14ms/step - loss: 3587.9285 - val_loss: 13192.0498
Epoch 7/1000
1/1 [==============================] - 0s 16ms/step - loss: 3547.9153 - val_loss: 13029.3281
Epoch 8/1000
1/1 [==============================] - 0s 14ms/step - loss: 3506.6123 - val_loss: 12863.3232
Epoch 9/1000
1/1 [==============================] - 0s 14ms/step - loss: 3464.3701 - val_loss: 12693.1680
Epoch 10/1000
1/1 [==========================

In [42]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: 153 || Predic: {test_output}")

Actual: 153 || Predic: [[152.89305]]


The result I got is 152.26 which is just a fraction short of the actual result. Therefore, we can conclude that for our dataset, bidirectional LSTM with single layer outperforms both the single layer and stacked unidirectional LSTMs.

## Many-to-one Sequence Problems with Multiple Features
In a many-to-one sequence problem we have an input where each time-steps consists of multiple features. The output can be a single value or multiple values, one per feature in the input time step. We will cover both the cases in this section.

## Creating the Dataset
Our dataset will contain 15 samples. Each sample will consist of 3 time-steps. Each time-steps will have two features.

Let's create two lists. One will contain multiples of 3 until 135 i.e. 45 elements in total. The second list will contain multiples of 5, from 1 to 225. The second list will also contain 45 elements in total. The following script creates these two lists:

In [46]:
X1 = np.array([x+3 for x in range(0, 135, 3)])
print(X1, '\n')

X2 = np.array([x+5 for x in range(0, 225, 5)])
print(X2)

[  3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54
  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 102 105 108
 111 114 117 120 123 126 129 132 135] 

[  5  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180
 185 190 195 200 205 210 215 220 225]


Each of the above list represents one feature in the time sample. The aggregated dataset can be created by joining the two lists as shown below:

In [48]:
X = np.column_stack((X1, X2))

We need to reshape our data into three dimensions so that it can be used by LSTM. We have 45 rows in total and two columns in our dataset. We will reshape our dataset into 15 samples, 3 time-steps, and two features.

In [50]:
X = array(X).reshape(15, 3, 2)
print(X)

[[[  3   5]
  [  6  10]
  [  9  15]]

 [[ 12  20]
  [ 15  25]
  [ 18  30]]

 [[ 21  35]
  [ 24  40]
  [ 27  45]]

 [[ 30  50]
  [ 33  55]
  [ 36  60]]

 [[ 39  65]
  [ 42  70]
  [ 45  75]]

 [[ 48  80]
  [ 51  85]
  [ 54  90]]

 [[ 57  95]
  [ 60 100]
  [ 63 105]]

 [[ 66 110]
  [ 69 115]
  [ 72 120]]

 [[ 75 125]
  [ 78 130]
  [ 81 135]]

 [[ 84 140]
  [ 87 145]
  [ 90 150]]

 [[ 93 155]
  [ 96 160]
  [ 99 165]]

 [[102 170]
  [105 175]
  [108 180]]

 [[111 185]
  [114 190]
  [117 195]]

 [[120 200]
  [123 205]
  [126 210]]

 [[129 215]
  [132 220]
  [135 225]]]


The output will also have 15 values corresponding to 15 input samples. Each value in the output will be the sum of the two feature values in the third time-step of each input sample. For instance, the third time-step of the first sample have features 9 and 15, hence the output will be 24. Similarly, the two feature values in the third time-step of the 2nd sample are 18 and 30; the corresponding output will be 48, and so on.

In [53]:
Y = np.array([ 24,  48,  72 , 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360])

Let's now solve this many-to-one sequence problem via simple, stacked, and bidirectional LSTMs.
## Solution via Simple LSTM

In [56]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 0s 128ms/step - loss: 30036.6953 - val_loss: 105264.6641
Epoch 2/1000
1/1 [==============================] - 0s 14ms/step - loss: 29504.6562 - val_loss: 103573.5625
Epoch 3/1000
1/1 [==============================] - 0s 14ms/step - loss: 29057.7656 - val_loss: 102303.0078
Epoch 4/1000
1/1 [==============================] - 0s 14ms/step - loss: 28671.8496 - val_loss: 101287.9297
Epoch 5/1000
1/1 [==============================] - 0s 18ms/step - loss: 28330.4004 - val_loss: 100429.1875
Epoch 6/1000
1/1 [==============================] - 0s 14ms/step - loss: 28021.1035 - val_loss: 99662.7266
Epoch 7/1000
1/1 [==============================] - 0s 13ms/step - loss: 27733.8184 - val_loss: 98943.0312
Epoch 8/1000
1/1 [==============================] - 0s 14ms/step - loss: 27459.5312 - val_loss: 98233.6797
Epoch 9/1000
1/1 [==============================] - 0s 14ms/step - loss: 27189.8125 - val_loss: 97500.4688
Epoch 10/1000
1/1 [============

The model is trained. We will create a test data point and then will use our model to make prediction on the test point.

The sum of two features of the third time-step of the input is 14 + 61 = 75. Our model with one LSTM layer predicted 73.41, which is pretty close.

In [57]:
test_input = array([[8, 51],
                    [11,56],
                    [14,61]])

test_input = test_input.reshape((1, 3, 2))
test_output = model.predict(test_input, verbose=0)
print(f"Actual: 75 || Prediction: {test_output}")

Actual: 75 || Prediction: [[83.77902]]


## Solution via Stacked LSTM
The following script trains a stacked LSTM and makes predictions on test point:

In [58]:
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(3, 2)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 0s 362ms/step - loss: 31196.2578 - val_loss: 113219.5000
Epoch 2/1000
1/1 [==============================] - 0s 14ms/step - loss: 31177.9121 - val_loss: 113162.4922
Epoch 3/1000
1/1 [==============================] - 0s 18ms/step - loss: 31164.1094 - val_loss: 112994.5000
Epoch 4/1000
1/1 [==============================] - 0s 14ms/step - loss: 31136.0723 - val_loss: 112467.0391
Epoch 5/1000
1/1 [==============================] - 0s 19ms/step - loss: 31059.7168 - val_loss: 111821.3984
Epoch 6/1000
1/1 [==============================] - 0s 15ms/step - loss: 30932.4082 - val_loss: 110881.0625
Epoch 7/1000
1/1 [==============================] - 0s 15ms/step - loss: 30718.3047 - val_loss: 109566.1875
Epoch 8/1000
1/1 [==============================] - 0s 17ms/step - loss: 30394.0293 - val_loss: 107318.2422
Epoch 9/1000
1/1 [==============================] - 0s 19ms/step - loss: 29882.1309 - val_loss: 104425.2578
Epoch 10/1000
1/1 [========

In [59]:

test_output = model.predict(test_input, verbose=0)
print(f"Actual: 75 || Prediction: {test_output}")

Actual: 75 || Prediction: [[76.253006]]


## Solution via Bidirectional LSTM
Here is the training script for simple bidirectional LSTM along with code that is used to make predictions on the test data point:

In [62]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(3, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 0s 197ms/step - loss: 24452.8281 - val_loss: 85734.0703
Epoch 2/1000
1/1 [==============================] - 0s 18ms/step - loss: 23926.5625 - val_loss: 83974.0312
Epoch 3/1000
1/1 [==============================] - 0s 13ms/step - loss: 23425.5156 - val_loss: 82206.8203
Epoch 4/1000
1/1 [==============================] - 0s 15ms/step - loss: 22935.8691 - val_loss: 80369.9531
Epoch 5/1000
1/1 [==============================] - 0s 16ms/step - loss: 22452.0488 - val_loss: 78610.9453
Epoch 6/1000
1/1 [==============================] - 0s 15ms/step - loss: 21980.1797 - val_loss: 77064.9766
Epoch 7/1000
1/1 [==============================] - 0s 15ms/step - loss: 21524.7793 - val_loss: 75646.7500
Epoch 8/1000
1/1 [==============================] - 0s 15ms/step - loss: 21077.5801 - val_loss: 74215.3203
Epoch 9/1000
1/1 [==============================] - 0s 15ms/step - loss: 20626.0762 - val_loss: 72675.0000
Epoch 10/1000
1/1 [=================

In [63]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: 75 || Prediction: {test_output}")

Actual: 75 || Prediction: [[80.28072]]


## Conclusion
Simple neural networks are not suitable for solving sequence problems since in sequence problems, in addition to current input, we need to keep track of the previous inputs as well. Neural Networks with some sort of memory are more suited to solving sequence problems. LSTM is one such network.

In this article, we saw how different variants of the LSTM algorithm can be used to solve one-to-one and many-to-one sequence problems. This is the first part of the article. In the second part, we will see how to solve one-to-many and many-to-many sequence problems. We will also study encoder-decoder mechanism that is most commonly used to create chatbots. Till then, happy coding :)